<img src="https://dutchanalytics.com/wp-content/uploads/2020/02/svg_logo_dutch_analytics-1.svg" width="80">

# Quickstart
Deploy a simple float-to-float model in UbiOps using the python client library.


The first step is to download a prepared model package (the model code).

In [ ]:
!curl -X GET https://storage.googleapis.com/ubiops/example-model-packages/example_model_package.zip -o simple_model_package.zip

Add your API token, provide a project name, model name and model version name. 
Afterward initialize the client library.

In [ ]:
API_TOKEN='Token <YOUR_API_KEY>'
PROJECT_NAME='<YOUR_PROJECT_NAME>'
MODEL_NAME='simple-tutorial'
MODEL_VERSION='v1'

In [ ]:
import ubiops
configuration = ubiops.Configuration()
configuration.api_key['Authorization'] = API_TOKEN
client = ubiops.ApiClient(configuration)
api = ubiops.CoreApi(client)
api.service_status()

## Deploy the model
Create a model.

In [ ]:
model_template = ubiops.ModelCreate(
    name=MODEL_NAME,
    description='A simple model that multiplies the input float by a random number.',
    input_type='structured',
    output_type='structured',
    input_fields=[ubiops.ModelInputFieldCreate(name='input', data_type='double')],
    output_fields=[ubiops.ModelOutputFieldCreate(name='output', data_type='double')]
)

model = api.models_create(project_name=PROJECT_NAME, data=model_template)
print(model)

Create a model version.

In [ ]:
model_version_template = ubiops.ModelVersionCreate(
    version=MODEL_VERSION,
    language='python3.7',
    memory_allocation=256,
    maximum_instances=1,
    minimum_instances=0,
    maximum_idle_time=1800 # = 30 minutes
)

model_version = api.model_versions_create(
    project_name=PROJECT_NAME,
    model_name=MODEL_NAME,
    data=model_version_template
)
print(model_version)

Upload the prepared model file to the created model version.

In [ ]:
file_upload_result =api.model_versions_file_upload(
    project_name=PROJECT_NAME,
    model_name=MODEL_NAME,
    version=MODEL_VERSION,
    file='simple_model_package.zip'
)

Check if model version is finished building.
This can take a few minutes.

In [ ]:
from time import sleep
status = 'building'
while status != 'available' and 'failed' not in status:    
    model_version_status = api.model_versions_get(       
        project_name=PROJECT_NAME,        
        model_name=MODEL_NAME,        
        version=MODEL_VERSION    
    )    
    status = model_version_status.status
    print(status, end='\r')
    sleep(1)
print(status)

## Creating a direct request
Make sure the model is in 'available' state before performing model requests.

In [ ]:
data = {'input': 123}
model_result = api.model_requests_create(
    project_name=PROJECT_NAME,
    model_name=MODEL_NAME,
    version=MODEL_VERSION,
    data=data
)
print(model_result)

***

# Pipelines

Multiple models can be chained using a *pipeline*. In this tutorial, we will chain the model created above with a new model. Just for demo purposes, we will use the same model package (the same code) for both models in the pipeline. Therefore, the pipeline will contain two modelling steps: each step will multiply the input by a random number.

## Create another model and version
Normally you would like to chain different models. However, just for demo purposes, we will use the same model package as used above.

Create the model and version

In [ ]:
MODEL_NAME2 = f"{MODEL_NAME}-2"
model_template2 = model_template
model_template2.name = MODEL_NAME2

api.models_create(
    project_name=PROJECT_NAME,
    data=model_template2
)

api.model_versions_create(
    project_name=PROJECT_NAME,
    model_name=MODEL_NAME2,
    data=model_version_template
)

Upload the zip

In [ ]:
api.model_versions_file_upload(
    project_name=PROJECT_NAME,
    model_name=MODEL_NAME2,
    version=MODEL_VERSION,
    file='simple_model_package.zip'
)

Wait for the model to be ready

In [ ]:
from time import sleep
status = 'building'
print(status, end='\r')
while status != 'available' and 'failed' not in status:    
    model_version_status = api.model_versions_get(       
        project_name=PROJECT_NAME,        
        model_name=MODEL_NAME2,        
        version=MODEL_VERSION    
    )    
    status = model_version_status.status
    print(status, end='\r')
    sleep(1)
print(status)

## Create a pipeline

In [ ]:
PIPELINE_NAME = f"{MODEL_NAME}-pipeline"

In [ ]:
pipeline_template = ubiops.PipelineCreate(
    name=PIPELINE_NAME,
    description='A simple pipeline that multiplies the input float by a random number.',
    input_type='structured',
    input_fields=[ubiops.ModelInputFieldCreate(name='input', data_type='double')]
)

api.pipelines_create(project_name=PROJECT_NAME, data=pipeline_template)

## Add models as objects to pipeline

In [ ]:
object_template = ubiops.PipelineObjectCreate(
    name=MODEL_NAME,
    reference_type='model',
    reference_name=MODEL_NAME,
    version=MODEL_VERSION
)
api.pipeline_objects_create(project_name=PROJECT_NAME, pipeline_name=PIPELINE_NAME, data=object_template)

In [ ]:
object_template2 = ubiops.PipelineObjectCreate(
    name=MODEL_NAME2,
    reference_type='model',
    reference_name=MODEL_NAME2,
    version=MODEL_VERSION
)
api.pipeline_objects_create(project_name=PROJECT_NAME, pipeline_name=PIPELINE_NAME, data=object_template2)

## Connect the objects

start -> model1

In [ ]:
connection_template = ubiops.AttachmentsCreate(
    source_name='pipeline_start', 
    destination_name=MODEL_NAME,
    mapping=[ubiops.AttachmentFieldsCreate(source_field_name='input', destination_field_name='input')]
)

api.pipeline_object_attachments_create(
    project_name=PROJECT_NAME, 
    pipeline_name=PIPELINE_NAME, 
    data=connection_template
)

model1 -> model2

In [ ]:
connection_template = ubiops.AttachmentsCreate(
    source_name=MODEL_NAME, 
    destination_name=MODEL_NAME2,
    mapping=[ubiops.AttachmentFieldsCreate(source_field_name='output', destination_field_name='input')]
)

api.pipeline_object_attachments_create(
    project_name=PROJECT_NAME, 
    pipeline_name=PIPELINE_NAME, 
    data=connection_template
)

## Creating a direct request

In [ ]:
data = {'input': 123}
pipeline_result = api.pipelines_request(
    project_name=PROJECT_NAME,
    pipeline_name=PIPELINE_NAME,
    data=data
)
print(pipeline_result)

## Cleanup
Delete created model.

In [ ]:
api.pipelines_delete(
    project_name=PROJECT_NAME,
    pipeline_name=PIPELINE_NAME
)
api.models_delete(
    project_name=PROJECT_NAME,
    model_name=MODEL_NAME
)
api.models_delete(
    project_name=PROJECT_NAME,
    model_name=MODEL_NAME2
)